In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
training_exp_name = "training/01_03_24_WMH_Ensemble"

dfc = rs.load_configs(
    root / training_exp_name,
    properties=False,
)

In [ ]:
dfc

In [ ]:
%%yaml inference_config

experiment:
    seed: 42

log:
    root: '?'
    log_interval: 50
    track_image_level: True 
    track_pixel_level: True 
    ignore_index: None 

model:
    exp_root : /storage/vbutoi/scratch/ESE/training/01_03_24_WMH_Ensemble
    checkpoint: "max-val-dice_score"
    ensemble: True

dataloader:
    batch_size: 8 
    num_workers: 1
    pin_memory: True 

dataset:
    split: '?' 
    slicing: dense_full 
    input_type: volume 

calibration:
    binarize: False 
    conf_interval_start: 0.5
    conf_interval_end: 1.0
    num_bins: 10
    neighborhood_width: 3
    square_diff: True

In [ ]:
%%yaml metrics_cfg 

qual_metrics:
    # - Edge_ECE:
    #     func: ese.experiment.metrics.ece.edge_ece_loss
    #     metric_type: calibration
    - ECW_ECE:
        func: ese.experiment.metrics.ece.ecw_ece_loss
        metric_type: calibration

cal_metrics:
    - ECE:
        func: ese.experiment.metrics.ece.ece_loss
    - CW_ECE:
        func: ese.experiment.metrics.ece.cw_ece_loss
    - ELM:
        func: ese.experiment.metrics.elm.elm_loss
    - CW_ELM:
        func: ese.experiment.metrics.elm.cw_elm_loss

In [ ]:
from ionpy.util import dict_product, Config
from ionpy.util.config import check_missing

# Need to define the experiment name
inference_exp_root = str(root / "inference/01_04_24_WMH_DebugEnsemble")

# Get the inference options.
##################################################
dataset_options = {
    'log.root': [inference_exp_root],
    'dataset.split': ['cal']
}

base_cfg = Config(inference_config).update(metrics_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    new_cfg = base_cfg.update(cfg_update)
    check_missing(new_cfg) # Verify there are no ? in config.
    cfgs.append(new_cfg)

In [ ]:
from ese.experiment.analysis.inference import get_cal_stats
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

get_cal_stats(cfgs[0])

In [ ]:
# from ionpy.slite.submit import submit_jobs
# from ese.experiment.analysis.inference import get_cal_stats

# submit_jobs(
#     exp_root=inference_exp_root,
#     config_list=cfgs, 
#     job_func=get_cal_stats,
#     available_gpus=["0"]
#     )